[![cloudevel](Imagenes/cloudevel.png)](https://cloudevel.com)

## El archivo ```Dockerfile```.

Docker nos permite crear nuestros propios contenedores mediante el uso de un archivo de configuración llamado ``Dockerfile``. Este archivo es un script, el cual permite crear un contenedor a partir de otro y realizar diversas operaciones con este.

### Comandos de Dockerfile:

* ```FROM``` Es el comando que importa la imagen base a partir de la cual se creará la nueva imagen.
* ```USER``` Define la UISD del usuario que será utilizado.
* ```VOLUME``` Liga a un directorio local con el contenedor.
* ```RUN``` La cual ejecuta diversos comandos al momento de la creación de la imagen.
* ```ADD``` Copia un archivo del sistema de archivos local al sistema de archivos de la imagen.
* ```ENV``` Define variables de entorno.
* ```CMD``` Ejecuta comandos al momento de crear un nuevo contenedor a partir de la imagen.
* ```ENTRYPOINT``` Define el comando que se utilizará por defecto al ejecutar un contenedor instanciado de la imagen.
* ```WORKDIR``` 
* ```EXPOSE``` define el reenvío de puertos.



## Construcción de una imagen a partir de un archivo ```Dockerfile```.

Para crear una imagen se utiliza el comando ```docker build``` de la siguiente forma:

```docker build -t <nombre de la imagen> <directorio de la Dockerfile>```

### Ejemplo del archivo Dockerfile.

* El archivo localizado en [src/Dockerfile](src/Dockerfile) contiene lo siguiente:

``` bash
#Descarga la imagen
FROM ubuntu:16.04
 
# Actualiza el sistema
RUN apt-get update
 
# Instala nginx, php-fpm y supervisord
RUN apt-get install -y nginx php7.0-fpm supervisor && \
    rm -rf /var/lib/apt/lists/*
 
#Define las variables de entorno
ENV nginx_vhost /etc/nginx/sites-available/default
ENV php_conf /etc/php/7.0/fpm/php.ini
ENV nginx_conf /etc/nginx/nginx.conf
ENV supervisor_conf /etc/supervisor/supervisord.conf
 
# habilita la configuración del servidor virtual de php-fpm en nginx
COPY default ${nginx_vhost}
RUN sed -i -e 's/;cgi.fix_pathinfo=1/cgi.fix_pathinfo=0/g' ${php_conf} && \
    echo "\ndaemon off;" >> ${nginx_conf}
 
#Copia la configuración del supervisor
COPY supervisord.conf ${supervisor_conf}
 
RUN mkdir -p /run/php && \
    chown -R www-data:www-data /var/www/html && \
    chown -R www-data:www-data /run/php
 
# Configura el volumen de almacenamiento
VOLUME ["/etc/nginx/sites-enabled", "/etc/nginx/certs", "/etc/nginx/conf.d", "/var/log/nginx", "/var/www/html"]
 
# Configura los servicios y los puertos 
COPY start.sh /start.sh
CMD ["./start.sh"]
 
EXPOSE 80 443
```

* El archivo [src/default]([src/default) contiene la configuración de nginx:

```
server {
    listen 80 default_server;
    listen [::]:80 default_server;
 
    root /var/www/html;
    index index.html index.htm index.nginx-debian.html;
 
    server_name _;
 
    location / {
        try_files $uri $uri/ =404;
    }
 
    location ~ \.php$ {
        include snippets/fastcgi-php.conf;
        fastcgi_pass unix:/run/php/php7.0-fpm.sock;
    }
 
    # deny access to .htaccess files, if Apache's document root
    # concurs with nginx's one
    #
    #location ~ /\.ht {
    #    deny all;
    #}
}
```

* El archivo [src/supervisord](src/supervisord) contiene lo siguiente:

```
[unix_http_server]
file=/dev/shm/supervisor.sock   ; (the path to the socket file)
 
[supervisord]
logfile=/var/log/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=false               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)
user=root             ;
 
; the below section must remain in the config file for RPC
; (supervisorctl/web interface) to work, additional interfaces may be
; added by defining them in separate rpcinterface: sections
[rpcinterface:supervisor]
supervisor.rpcinterface_factory = supervisor.rpcinterface:make_main_rpcinterface
 
[supervisorctl]
serverurl=unix:///dev/shm/supervisor.sock ; use a unix:// URL  for a unix socket
 
; The [include] section can just contain the "files" setting.  This
; setting can list multiple files (separated by whitespace or
; newlines).  It can also contain wildcards.  The filenames are
; interpreted as relative to this file.  Included files *cannot*
; include files themselves.
 
[include]
files = /etc/supervisor/conf.d/*.conf
 
 
[program:php-fpm7.0]
command=/usr/sbin/php-fpm7.0 -F
numprocs=1
autostart=true
autorestart=true
 
[program:nginx]
command=/usr/sbin/nginx
numprocs=1
autostart=true
autorestart=true
```

* El archivo [src/start.sh](src/supervisord) tiene permisos de ejecución y contiene lo siguiente:

```
#!/bin/sh
 
/usr/bin/supervisord -n -c /etc/supervisor/supervisord.conf
```

### Creación de la imagen a partir del Dockerfile.

In [ ]:
cd src

In [ ]:
pwd

In [ ]:
sudo docker build -t nginx_image .

In [ ]:
sudo docker images

### Creación de un contenedor a partir de la nueva imagen.

In [ ]:
mkdir webroot

In [ ]:
sudo systemctl stop httpd

In [ ]:
sudo docker run -d -v webroot:/var/www/html -p 80:80 --name servidor nginx_image

* Si se está ejecutando  esta notebook direactamente desde su equipo, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost
* Si se está ejecutando  esta notebook desde la VM provista del sitio de Cloudevel®, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost:8980

In [ ]:
sudo docker ps

In [ ]:
sudo docker stop servidor

In [ ]:
sudo docker ps -a

In [ ]:
sudo docker rm servidor

In [ ]:
sudo docker ps -a

In [ ]:
sudo docker images

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>